In [1]:
############
## This script read metadata from netCDF file, convert it into rdf triple store, outputs/data.ttl, outputs/data.trig
## Xiaofeng Liao
## 25 Feb, 2020
###############

In [1]:
pip install netCDF4
pip install rdflib
pip install PyYAML

SyntaxError: invalid syntax (<ipython-input-1-102418f64239>, line 1)

In [2]:
import yaml
import netCDF4
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef, plugin, ConjunctiveGraph
from rdflib.namespace import DC, FOAF, RDFS
from rdflib.store import Store
import glob


In [3]:
#import nltk; nltk.download('all')

In [4]:
ae = Namespace("http://actrisexample.eu/ns/")

vocab = dict()

store = plugin.get('IOMemory', Store)()

g = ConjunctiveGraph(store)
g.bind('envri', 'http://envri.eu/ns/')
g.bind('ae','http://actrisexample.eu/ns/')
g.bind('dcterms', 'http://purl.org/dc/terms/')
#g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.bind('skos', 'http://www.w3.org/2004/02/skos/core#')

with open('vocab.yaml', 'r') as f:
    for key, value in yaml.safe_load(f).items():
        v = dict()
        if key == 'Component':
            uris = eval(str(value['uri']))
            for uri in uris:
                #print(uri)
                v['uri'] = URIRef(uri)
                v['label'] = value['label'].replace(' ','_')
                #print(value['label'])
                g.add(( v['uri'], RDFS.label, Literal(v['label']) ))
                g.add( ( v['uri'], URIRef('http://actrisexample.eu/ns/hasvalue'), Literal('aerosol_absorption_coefficient')) )
        else:
            v['uri'] = URIRef(value['uri'])
            #print(v['uri'])
            v['label'] = value['label'].replace(' ','_')
            #print(v['label'])
            vocab[key] = v
            g.add((vocab[key]['uri'], RDFS.label, Literal(vocab[key]['label'])))

print('Instrument filter_absorption_photometer and variable aerosol_absorption_coefficient:\nFilename: DK0025G.20030101000000.20170713083448.filter_absorption_photometer.aerosol_absorption_coefficient.aerosol.12y.1h.US08L_Magee_AE16_SUM.US08L_aethalometer.lev2.nc\n\n')

# use the netcdf4 lib to extract netcdf via the opendap protocol
dataset = netCDF4.Dataset('DK0025G.20030101000000.20170713083448.filter_absorption_photometer.aerosol_absorption_coefficient.aerosol.12y.1h.US08L_Magee_AE16_SUM.US08L_aethalometer.lev2.nc','r','NETCDF4')
# fetch global attributes from netcdf on the thredds server
global_attr = dataset.ncattrs()



## Following lines build nodes for instrument filter_absorption_photometer and its measuring variable aerosol_absorption_coefficient
## and link them with ae.measures
## this should be automated later
filter_absorption_photometer = BNode()
aerosol_absorption_coefficient = BNode()

g.add( (filter_absorption_photometer, RDFS.label, Literal('filter_absorption_photometer') ) )
g.add( (aerosol_absorption_coefficient, RDFS.label, Literal('aerosol_absorption_coefficient') ) )
g.add( (filter_absorption_photometer, ae.measures, aerosol_absorption_coefficient) )



for attr in global_attr:
    if(attr == "ebas_metadata"):
        ebas_metadata = eval(dataset.getncattr(attr))
        for k,v in ebas_metadata.items():
            print('{0}: {1}'.format(k, v))
            if (k !="Component"):
                #uris = eval(str(vocab[k]['uri']))
                #for item in uris:
                #    #print(item)
                #    g.add( ( URIRef(item), vocab['hasvalue']['uri'], Literal(v)) )
            #else:
                g.add( ( URIRef(vocab[k]['uri']), vocab['hasvalue']['uri'], Literal(v.replace(' ','_'))) )
    else:
        print('{0}: {1}'.format(attr, dataset.getncattr(attr)))
        g.add( (URIRef(vocab[attr]['uri']), vocab['hasvalue']['uri'], Literal(str(dataset.getncattr(attr)).replace(' ','_'))) )

print("\n--- printing raw triples ---")
for s, p, o in g:
    print('\n')
    print((s, p, o))
    

print("\n graph has %s statements." % len(g))

g.serialize(destination='outputs/data.ttl', format='ttl')
g.serialize(destination='outputs/data.trig', format='trig')

Instrument filter_absorption_photometer and variable aerosol_absorption_coefficient:
Filename: DK0025G.20030101000000.20170713083448.filter_absorption_photometer.aerosol_absorption_coefficient.aerosol.12y.1h.US08L_Magee_AE16_SUM.US08L_aethalometer.lev2.nc


Conventions: CF-1.7, ACDD-1.3
featureType: timeSeries
title: Ground based in situ observations of aerosol_absorption_coefficient at Summit (DK0025G) using filter_absorption_photometer
keywords: NOAA-ESRL, DK0025G, aerosol_absorption_coefficient, Summit, aerosol, GAW-WDCA, volume_absorption_coefficient_in_air_due_to_dried_aerosol_particles
id: DK0025G.20030101000000.20170713083448.filter_absorption_photometer.aerosol_absorption_coefficient.aerosol.12y.1h.US08L_Magee_AE16_SUM.US08L_aethalometer.lev2.nc
naming_authority: no.nilu.ebas
project: NOAA-ESRL, GAW-WDCA
acknowledgement: Request acknowledgement details from data originator
license: NOAA-ESRL: , GAW-WDCA: 
summary: Ground based in situ observations of aerosol_absorption_coeffici